# Get the lyric texts in chorus sections

In [7]:
import pandas as pd
import os

def _str_2_sec(lrc_time):
    min_sec = lrc_time.split(':')
    sec = float(min_sec[0])*60 + float(min_sec[1])
    return sec

def _parse_lrcline(lrc):
    lrc_t_w = lrc.split(']')
    if lrc_t_w[0][1].isdecimal(): # has timestamp
        if len(lrc_t_w) > 1:
            if not ('作' in lrc_t_w[1]): # lrc sentence
                lrc_time = _str_2_sec(lrc_t_w[0][1:])
                return lrc_time, lrc_t_w[1]
    return None

def parse_lrc_text(music_id, lrc):
    lrc_chorus = ''
#     record = meta.loc[music_id]
#     start_time = _str_2_sec(record['chorus_start_time'])
#     end_time = _str_2_sec(record['chorus_end_time'])
    with open(lrc) as f:
        lrc_string = ''.join(f.readlines()).strip()
    for lrc_line in lrc_string.split('\n'):
        lrc_t_w = _parse_lrcline(lrc_line)
        if lrc_t_w:
#             if lrc_t_w[0] > start_time and lrc_t_w[0] < end_time:
            lrc_chorus = lrc_chorus+lrc_t_w[1]+'. '
    return lrc_chorus.strip()

In [2]:
lrc_dir = '/Users/huizhang/Desktop/PMEmo2.0/dataset2.0/lyrics'
lrcs = [f for f in os.listdir(lrc_dir) if f[-4:]=='.lrc']
def get_id(s):
    return int(s[:-4])
df = pd.DataFrame(index=map(get_id, lrcs), columns = ['lrc_text'])
df.index.name ='musicId'

In [18]:
for lrc in lrcs:
    lrc_path = os.path.join(lrc_dir, lrc)
    music_id = get_id(lrc)
    lrc_text = parse_lrc_text(music_id, lrc_path)
    df.loc[music_id] = lrc_text
    
df = df[df['lrc_text'] != '']

In [27]:
static_anno = pd.read_csv('static_annotations.csv', index_col=0)
dataset = df.join(static_anno).dropna()
dataset.to_csv('lrc_dataset.csv')